# Example: the PanTaGruEl model of the European high-voltage network

This notebook provides a complete example of how synthetic time series can be imported into a network given in the [PowerModel.jl](https://github.com/lanl-ansi/PowerModels.jl) format. 

To begin with, import the packages PowerModels and TemperateOptimalPowerFlow (TOPF), as well as a few other convenient packages:

In [1]:
using PowerModels, TemperateOptimalPowerFlow

Precompiling TemperateOptimalPowerFlow
  ✓ TemperateOptimalPowerFlow
  1 dependency successfully precompiled in 3 seconds. 80 already precompiled.


In [2]:
using DataFrames, CSV

## Prepare the PanTaGruEl model

The PanTaGruEl network can be imported from the file `models/pantagruel.json` generated using the package [PanTaGruEl.jl](https://github.com/laurentpagnier/PanTaGruEl.jl).

In [3]:
network = parse_file("../models/pantagruel.json")

[info | PowerModels]: removing 2 cost terms from generator 788: Any[]
[info | PowerModels]: removing 2 cost terms from generator 1070: Any[]
[info | PowerModels]: removing 2 cost terms from generator 1067: Any[]
[info | PowerModels]: removing 2 cost terms from generator 1068: Any[]
[info | PowerModels]: removing 2 cost terms from generator 1069: Any[]
[info | PowerModels]: removing 2 cost terms from generator 1066: Any[]
[info | PowerModels]: removing 2 cost terms from generator 789: Any[]


Dict{String, Any} with 11 entries:
  "bus"      => Dict{String, Any}("4304"=>Dict{String, Any}("coord"=>Any[11.843…
  "name"     => "pantagruel"
  "dcline"   => Dict{String, Any}()
  "gen"      => Dict{String, Any}("1"=>Dict{String, Any}("model"=>2, "gen_bus"=…
  "branch"   => Dict{String, Any}("4304"=>Dict{String, Any}("br_r"=>0.00174284,…
  "storage"  => Dict{String, Any}()
  "switch"   => Dict{String, Any}()
  "baseMVA"  => 100
  "per_unit" => true
  "shunt"    => Dict{String, Any}()
  "load"     => Dict{String, Any}("3935"=>Dict{String, Any}("status"=>1, "load_…

The first step is to create lists that will be used as a static reference for all loads and generators.

In [4]:
list_of_loads = create_list_of_loads(network)
length(list_of_loads)

3998

In [5]:
list_of_gens = create_list_of_gens(network)
length(list_of_gens)

1083

These lists should remain the same through all the process of data generation.
If some files have already been created to store them (see below), then we should load the lists from the files instead:

In [6]:
list_of_loads = string.(CSV.read("data/pantagruel_load_info.csv", DataFrame).id)
list_of_gens = string.(CSV.read("data/pantagruel_gen_info.csv", DataFrame).id);

Then we can gather some information for all loads in the list, such as the country label and the relative weight of the load in that country:

In [7]:
loads_info = get_loads_info(network, list_of_loads, ["country", "load_prop", "name"])

Row,id,country,load_prop,name
,String,Any,Any,Any
1,3935,DE,0.0022038,Unnamed 1340
2,2243,FR,0.000443208,Vallorcine
3,1881,FR,0.00127502,Mallemort
4,1907,FR,0.000635447,Breuil
5,2923,FR,0.000650249,La Justice
6,599,ES,0.000506166,Alcobendas
7,2491,IT,0.00328872,Trav.
8,228,ES,0.00773441,Sant Fost
9,2590,NL,0.0248207,Zwolle


There are 22 countries in the current version of PanTaGruEl:

In [8]:
countries = unique(loads_info[:, "country"])
length(countries)

22

The number of loads in each country varies:

In [9]:
load_count_by_country = Dict(country => count(x->(x == country), loads_info[:, "country"]) for country ∈ countries)
sort(load_count_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Int64} with 22 entries:
  "ES" => 908
  "FR" => 843
  "DE" => 560
  "IT" => 323
  "PL" => 188
  "DK" => 180
  "CH" => 163
  "PT" => 145
  "RO" => 124
  "AT" => 78
  "CZ" => 71
  "BG" => 64
  "BE" => 50
  "HU" => 47
  "RS" => 47
  "SK" => 43
  "GR" => 38
  "NL" => 38
  "BA" => 35
  "HR" => 28
  "SI" => 15
  "ME" => 10

We can verify that the weights of the loads add up to one for each country separately:

In [10]:
Dict(country => sum(loads_info[loads_info[:, "country"] .== country, "load_prop"]) for country ∈ countries)

Dict{String, Float64} with 22 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "HU" => 1.0
  "CH" => 1.0
  "RS" => 1.0
  "DE" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.0
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.0
  "BE" => 1.0
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "BA" => 1.0
  "DK" => 1.0

The generators can also be examined:

In [11]:
gens_info = get_gens_info(network, list_of_gens, ["type", "pmax", "country", "name", "cost"])

Row,id,type,pmax,country,name,cost
,String,Any,Any,Any,Any,Any
1,1,Hydro,1.14,BG,Uzundzhovo,"Any[6000, 0]"
2,519,hydro_pure_ps,11.64,BE,Coo,"Any[10000, 0]"
3,788,Geothermal,0.19,IT,Montalto,Any[]
4,774,fossil_brown_lignite,1.77,HU,Tisza II,"Any[2400, 0]"
5,599,Biomass,0.01902,FR,Beaucouze,"Any[1000, 0]"
6,491,hydro_ror,0.723,AT,Pyhrn,"Any[1000, 0]"
7,228,Hydro,0.164,DE,Wengerohr,"Any[6000, 0]"
8,332,fossil_coal_hard,2.46,PL,Poznań Południe,"Any[3500, 0]"
9,190,fossil_brown_lignite,2.52,CZ,Tisová,"Any[2400, 0]"


Note that the cost is always linear in PanTaGruEl, so the "cost" column can be replaced by its linear value:

In [12]:
gens_info.cost = (x->length(x) > 0 ? x[1] : 0).(gens_info.cost)
gens_info

Row,id,type,pmax,country,name,cost
,String,Any,Any,Any,Any,Int64
1,1,Hydro,1.14,BG,Uzundzhovo,6000
2,519,hydro_pure_ps,11.64,BE,Coo,10000
3,788,Geothermal,0.19,IT,Montalto,0
4,774,fossil_brown_lignite,1.77,HU,Tisza II,2400
5,599,Biomass,0.01902,FR,Beaucouze,1000
6,491,hydro_ror,0.723,AT,Pyhrn,1000
7,228,Hydro,0.164,DE,Wengerohr,6000
8,332,fossil_coal_hard,2.46,PL,Poznań Południe,3500
9,190,fossil_brown_lignite,2.52,CZ,Tisová,2400


The info tables can be exported to CSV files:

In [13]:
# CSV.write("$data_directory/pantagruel_load_info.csv", loads_info)

In [14]:
# CSV.write("$data_directory/pantagruel_gen_info.csv", gens_info)

The file `pantagruel_loads.csv` can then be used as a basis to generate time series for the loads using the `TimeSeries` package.
Similarly, the file `pantagruel_gens.csv` can be used as a basis to generate a variable production cost. 

The documentation notebook [PanTaGruEl_time_series.ipynb](./PanTaGruEl_time_series.ipynb) shows how to proceed and creates CSV files with synthetic time series named `pantagruel_load_series.csv` and `pantagruel_gen_cost_series.csv`

## Assign loads corresponding to a given timestep

The model comes with a pre-set value for each load:

In [15]:
[network["load"][string(id)]["pd"] for id = 1:10]

10-element Vector{Float64}:
 0.991773959053433
 0.8502485878481437
 0.5491873376602063
 0.42357239643070443
 1.7639757764158501
 1.0707845577547312
 1.8966499644786388
 1.7682447350743928
 2.6672019091946333
 0.5240039676629611

These values can be replaced by any particular timestep of the synthetic time series:

In [16]:
assign_loads_from_file!(network, "data/pantagruel_load_series.csv", 0)
[network["load"][string(id)]["pd"] for id = 1:10]

10-element Vector{Float64}:
 0.7446595719725443
 0.5464496727856487
 0.3541105263040253
 0.38418819310111674
 1.0846321773314256
 0.7425666094818514
 1.5055856572304538
 1.0089063917456256
 1.652165996556241
 0.3566745501057052

In some cases, it is more convenient to load all the time series in memory first:

In [17]:
loads = CSV.read("data/pantagruel_load_series.csv", DataFrame);

Then the load assignment works much faster. For instance, one can compare a winter day at peak hour:

In [18]:
assign_loads!(network, loads, 17)
[network["load"][string(id)]["pd"] for id = 1:10]

10-element Vector{Float64}:
 1.0345068969732787
 0.8260448280655366
 0.5460505702683331
 0.5049749730894599
 1.8274297373471924
 1.2671350244379402
 2.443172673713758
 1.6437777493464163
 2.7921392822040714
 0.47924970577535086

...with a quiet summer night where the loads are much smaller:

In [19]:
assign_loads!(network, loads, 24*182)
[network["load"][string(id)]["pd"] for id = 1:10]

10-element Vector{Float64}:
 0.5375472554440054
 0.47201134948025997
 0.31346646960682395
 0.23781537063908323
 1.3216694606077157
 0.8042222569025296
 1.1033609072302852
 1.3706421892906258
 1.4662215546392
 0.3296066215708211

## Assign production costs corresponding to a given timestep

As with the load, the model already has a cost value for every generator:

In [20]:
[network["gen"][string(id)]["cost"] for id = 1:10]

10-element Vector{Vector{Any}}:
 [6000, 0]
 [10000, 0]
 [1000, 0]
 [1000, 0]
 [6000, 0]
 [8000, 0]
 [8000, 0]
 [11000, 0]
 [2400, 0]
 [2400, 0]

These values can be replaced with a particular timestep of some time series, either reading directly from a file:

In [21]:
assign_costs_from_file!(network, "data/pantagruel_gen_cost_series.csv", 0)
[network["gen"][string(id)]["cost"] for id = 1:10]

10-element Vector{Vector{Any}}:
 [6420.331385689227, 0]
 [9832.759503044084, 0]
 [999.4252626883483, 0]
 [1061.43632511772, 0]
 [5942.199680823749, 0]
 [7787.958883076765, 0]
 [8394.24526193362, 0]
 [11491.79281589861, 0]
 [2322.4650601250573, 0]
 [2363.4118696171918, 0]

...or loading the entire time series into memory first:

In [22]:
costs = CSV.read("data/pantagruel_gen_cost_series.csv", DataFrame);

In [23]:
assign_costs!(network, costs, 1)
[network["gen"][string(id)]["cost"] for id = 1:10]

10-element Vector{Vector{Any}}:
 [7164.09240275266, 0]
 [10110.966350775965, 0]
 [945.8202678869371, 0]
 [1015.5867487647798, 0]
 [5800.825047859604, 0]
 [7942.79244515956, 0]
 [8219.912463126351, 0]
 [10917.159462393794, 0]
 [2292.4431070389596, 0]
 [2648.4385455151587, 0]

## Perform Optimal Power Flow computation

This requires using an optimizer, such as Ipopt (free) or Gurobi (commercial):

In [24]:
# using Ipopt
using Gurobi

Precompiling TemperateOptimalPowerFlowGurobiExt


Set parameter TokenServer to value "gurobilm.hevs.ch"


  ✓ TemperateOptimalPowerFlow → TemperateOptimalPowerFlowGurobiExt
  1 dependency successfully precompiled in 2 seconds. 83 already precompiled.


An optimizer can be obtained with:

In [25]:
optimizer = get_optimizer();

Assign loads and generation costs from a particular time step:

In [26]:
assign_loads!(network, loads, 0)
assign_costs!(network, costs, 0)

The command to run a DC OPF taking line costs into account is:

In [27]:
opf = solve_dc_topf(network, optimizer)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 23081 rows, 13555 columns and 64175 nonzeros
Model fingerprint: 0x3c7b231e
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [7e+02, 1e+04]
  Bounds range     [1e-02, 5e+01]
  RHS range        [7e-12, 7e+00]
Presolve removed 16605 rows and 7678 columns
Presolve time: 0.05s
Presolved: 6476 rows, 7549 columns, 29157 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Free vars  : 518
 AA' NZ     : 7.154e+04
 Factor NZ  : 2.406e+05 (roughly 8 MB of memory)
 Factor Ops : 1.245e+07 (less than 1 second per iteration)
 Threads    : 10

                  Objective                Residual
Iter       Primal          Dua

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.191429
  "optimizer"          => "Gurobi"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 4.07763e6
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => 4.07763e6

The generators' power can then be read from the OPF's solution:

In [28]:
[opf["solution"]["gen"][gen_id]["pg"] for gen_id in list_of_gens]

1083-element Vector{Float64}:
  0.0
  0.0
  0.19
  1.77
  0.01902
  0.72300003
  0.0
  0.0
  2.52
  0.0
  0.0
  0.0
  0.93
  ⋮
  0.0
  1.38
  0.96
  0.0
  0.0
  0.93
  0.0
 52.0
  0.0
  4.08
  0.0
  0.0

Other information, such as the power flowing through a line can also be obtained:

In [29]:
[opf["solution"]["branch"][string(i)]["pt"] for i = 1:10]

10-element Vector{Float64}:
  0.3723297859862722
  0.3723297859862722
  0.02577313102452905
  0.02577313102452905
  0.4949034107365913
 -0.5258894736959743
 -0.21581180689888324
  0.15148139938030436
 -0.15148139938030436
  0.0

In some cases, the OPF won't converge, and one may want to raise the thermal limit, for instance by 10% with the function:

In [30]:
raise_thermal_limit!(network, 1.1)

To lower the thermal limit again, re-load the network or use:

In [31]:
raise_thermal_limit!(network, 1/1.1)

## Implement ramp up/down constraints

Ramp up/down constraints take into account the solution to a previous OPF to bound the possible active power of some generators.

To use it, the network data must first be updated to include the results of the OPF computation:

In [32]:
update_data!(network, opf["solution"])

In [33]:
opf = solve_dc_topf(network, optimizer)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 23081 rows, 13555 columns and 64175 nonzeros
Model fingerprint: 0xec622379
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [7e+02, 1e+04]
  Bounds range     [1e-02, 5e+01]
  RHS range        [7e-12, 7e+00]
Presolve removed 16605 rows and 7678 columns
Presolve time: 0.04s
Presolved: 6476 rows, 7549 columns, 29157 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Free vars  : 518
 AA' NZ     : 7.154e+04
 Factor NZ  : 2.406e+05 (roughly 8 MB of memory)
 Factor Ops : 1.245e+07 (less than 1 second per iteration)
 Threads    : 10

                  Objective                Residual
Iter       Primal          Dua

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.212495
  "optimizer"          => "Gurobi"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 4.07763e6
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => 4.07763e6

Then, one can assign a maximum ramp up/down value for generators of a given type (in per units):

In [34]:
assign_ramp_max!(network, 1.0, ["nuclear", "Nuclear", "nuclear_cons"])

In [35]:
network["gen"]["257"]

Dict{String, Any} with 16 entries:
  "ramp_max"   => 1.0
  "pg"         => 12.88
  "model"      => 2
  "qg"         => NaN
  "gen_bus"    => 2807
  "pmax"       => 12.88
  "mbase"      => 100
  "vg"         => 1
  "index"      => 257
  "cost"       => Any[1700.98, 0]
  "gen_status" => 1
  "qmax"       => 6.44
  "qmin"       => -6.44
  "type"       => "nuclear"
  "pmin"       => 0
  "ncost"      => 2

This can also be done as a fraction of the max capacity:

In [36]:
assign_ramp_max_ratio!(network, 0.1, ["nuclear", "Nuclear", "nuclear_cons"])

In [37]:
network["gen"]["257"]

Dict{String, Any} with 16 entries:
  "ramp_max"   => 1.288
  "pg"         => 12.88
  "model"      => 2
  "qg"         => NaN
  "gen_bus"    => 2807
  "pmax"       => 12.88
  "mbase"      => 100
  "vg"         => 1
  "index"      => 257
  "cost"       => Any[1700.98, 0]
  "gen_status" => 1
  "qmax"       => 6.44
  "qmin"       => -6.44
  "type"       => "nuclear"
  "pmin"       => 0
  "ncost"      => 2

Assign loads and line costs from the next time step:

In [38]:
assign_loads!(network, loads, 1)
assign_costs!(network, costs, 1)

Perform a new OPF (note that the number of constraints has changed):

In [39]:
opf = solve_dc_topf(network, optimizer)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 23081 rows, 13555 columns and 64175 nonzeros
Model fingerprint: 0x922be90a
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [7e+02, 1e+04]
  Bounds range     [1e-02, 5e+01]
  RHS range        [7e-12, 7e+00]
Presolve removed 16605 rows and 7678 columns
Presolve time: 0.04s
Presolved: 6476 rows, 7548 columns, 29156 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Free vars  : 518
 AA' NZ     : 7.154e+04
 Factor NZ  : 2.408e+05 (roughly 8 MB of memory)
 Factor Ops : 1.247e+07 (less than 1 second per iteration)
 Threads    : 10

                  Objective                Residual
Iter       Primal          Dua

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.195692
  "optimizer"          => "Gurobi"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 3.89197e6
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => 3.89197e6

## Iterate OPF

To iterate the process for every time step, one can use the function `iterate_dc_opf`:

In [40]:
# iterate_dc_opf(network,
#     "data/pantagruel_load_series.csv", 
#     "data/pantagruel_gen_cost_series.csv",
#     "data/pantagruel_gen_series.csv",
#     get_silent_optimizer())

## Add line costs

In general, the OPF results in a certain number of lines being loaded at their maximum rate:

In [41]:
optimizer = get_silent_optimizer();

In [42]:
line_rates = compute_line_rates(network, optimizer)

Dict{String, Float64} with 8375 entries:
  "4304" => 0.039106
  "5422" => 0.0191124
  "3935" => 0.403133
  "5461" => 0.110672
  "2243" => 0.0197504
  "8169" => 0.330046
  "1881" => 0.0507151
  "5425" => 0.0172724
  "4209" => 0.427278
  "1907" => 0.138911
  "2923" => 0.409635
  "6753" => 0.137247
  "7413" => 0.12305
  "599"  => 0.110428
  "2491" => 0.0824372
  "5944" => 0.0767605
  "228"  => 0.0160537
  "2590" => 1.0
  "3697" => 0.0208966
  "5031" => 0.10786
  "2579" => 0.196347
  "5551" => 0.366423
  "3991" => 0.0293508
  "2562" => 0.0877315
  "3215" => 0.545515
  ⋮      => ⋮

In [43]:
length(line_rates)

8375

In [44]:
count(values(line_rates) .> 0.99)

39

Adding a cost for the line in the objective function solves the problem:

In [45]:
add_line_costs!(network, 5000)

In [46]:
improved_line_rates = compute_line_rates(network, optimizer)
count(values(improved_line_rates) .> 0.99)

1

**More information about line costs can be found in the notebook [PanTaGruEl_line_rates](./PanTaGruEl_line_rates.ipynb).**